# Fiber Activity Detection

Extract fluorescence activity from neurite-like structures in a timelapse ScanImage TIFF.

Uses `mbo_utilities.imread` to lazily load the raw data, then runs the
fiber activity pipeline to segment fibres, extract ROI traces, and filter
by activity threshold.

In [1]:
!uv pip install jupyterlab-vim

Using Python 3.12.12 environment at: C:\Users\flynn\repos\mbo_utilities\.venv
Audited 1 package in 33ms


In [2]:
import numpy as np
from pathlib import Path
import mbo_utilities as mbo
from mbo_utilities.analysis.fiber_activity import run, FiberActivityResult

## 1. Load timelapse data

Counting frames:   0%|          | 0/6 [00:00<?, ?it/s]

Shape: (1163, 14, 448, 448), dtype: int16


## 2. Select a single z-plane (if volumetric)

The fiber activity pipeline expects a single-plane timelapse `(T, Y, X)`.
If your data is volumetric `(T, Z, Y, X)`, select a single plane here.

In [13]:
data_path = r"D:/SERVER_DATA/raw_scanimage_tiffs"
data = mbo.imread(data_path)
print(f"Shape: {data.shape}, dtype: {data.dtype}")

outpath = Path(data_path).joinpath("out")
outpath.mkdir(exist_ok=True)
out = mbo.imwrite(data, outpath, overwrite=True, planes=[1, 7, 14])

Non-uniform z-plane spacing detected (steps: [6, 7]...). Using first step (6) for dz calculation.


Writing TIFF:   0%|          | 0/3489 [00:00<?, ?pg/s]

In [14]:
out

WindowsPath('D:/SERVER_DATA/raw_scanimage_tiffs/out/tp00001-01163_zplane01-14_stack.tif')

## 3. Run fiber activity pipeline

In [9]:
outpath = Path(data_path).joinpath("segmented")
outpath.mkdir(exist_ok=True)

In [16]:
arr = mbo.imread(out)
arr.shape

(1163, 3, 448, 448)

In [15]:
result = run(
    out,
    fiber_thickness=3,
    percentile_threshold=95.0,
    min_object_size=4,
    activity_threshold=0.2,
    frame_rate=17.58,
    output_dir=Path(data_path).parent / "fiber_activity_results",
)

print(f"Found {result.traces.shape[1]} active ROIs across {result.traces.shape[0]} frames")

RuntimeError: weights.ndim (2) must match len(axes) (3)

## 4. Visualise segmentation

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(result.mean_image, cmap="gray")
axes[0].set_title("Mean image")
axes[0].axis("off")

axes[1].imshow(result.enhanced_image, cmap="gray")
axes[1].set_title("Enhanced (fibre filter)")
axes[1].axis("off")

axes[2].imshow(result.mean_image, cmap="gray")
axes[2].imshow(result.binary_mask, cmap="Reds", alpha=0.4)
axes[2].set_title("Segmentation mask overlay")
axes[2].axis("off")

plt.tight_layout()
plt.show()

## 5. Spatial ROI map

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(result.mean_image, cmap="gray")
ax.scatter(
    result.locations[:, 0],
    result.locations[:, 1],
    s=40, c="red", edgecolors="yellow", linewidths=0.8,
)
for i in range(len(result.locations)):
    ax.text(
        result.locations[i, 0] + 4,
        result.locations[i, 1] + 4,
        str(i + 1), color="yellow", fontsize=8, fontweight="bold",
    )
ax.set_title(f"{len(result.locations)} active ROIs")
ax.axis("off")
plt.show()

## 6. Stacked activity traces

In [ ]:
n_rois = result.traces.shape[1]
n_frames = result.traces.shape[0]
time_vector = np.arange(n_frames) / result.frame_rate

spacing = np.percentile(result.traces, 99) * 0.8
traces_per_fig = 10

for fig_idx in range(0, n_rois, traces_per_fig):
    fig, ax = plt.subplots(figsize=(12, 6))
    batch = range(fig_idx, min(fig_idx + traces_per_fig, n_rois))
    for j, roi_idx in enumerate(batch):
        offset = j * spacing
        ax.plot(time_vector, result.traces[:, roi_idx] + offset, linewidth=1)
        ax.text(
            time_vector[0], offset + result.traces[:, roi_idx].mean(),
            f"ROI {roi_idx + 1}", ha="right", fontsize=8,
        )
    ax.set_title(f"Activity traces (ROIs {batch[0]+1}–{batch[-1]+1})")
    ax.set_xlabel("Time (s)")
    ax.set_yticks([])
    plt.tight_layout()
    plt.show()